In [1]:
%pip install langchain langchain_community


[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pymupdf
%pip install --upgrade --quiet  langchain sentence_transformers
%pip install langchain_huggingface
%pip install langchain-chroma


[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("dataset.pdf")
data = loader.load()
# print(data[0])

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(data)

In [7]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="distilbert-base-uncased")

No sentence-transformers model found with name distilbert-base-uncased. Creating a new one with mean pooling.


In [8]:
from langchain.vectorstores.chroma import Chroma

db = Chroma.from_documents(chunks, embeddings)

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context and only provide sentences relevant to the question. Do not provide additional sentences: {question}
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

qa_model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(qa_model_name)
qa_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

def answer_question(query):
    docs = db.similarity_search(query, k=1)
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = prompt_template.format(context=context, question=query)

    response = qa_pipeline(prompt)
    response_text = response[0]['summary_text']

    formatted_response = f"Response: {response_text}\nSources: {docs}"
    return formatted_response

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
